<a href="https://colab.research.google.com/github/anuthereaper/PythonLibrary/blob/main/PGP_encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pgpy

In [ ]:
import pgpy
from pgpy.constants import PubKeyAlgorithm, KeyFlags, HashAlgorithm, SymmetricKeyAlgorithm, CompressionAlgorithm
from timeit import default_timer as timer
import base64

# we can start by generating a primary key. For this example, we'll use RSA, but it could be DSA or ECDSA as well
key = pgpy.PGPKey.new(PubKeyAlgorithm.RSAEncryptOrSign, 4096)
# we now have some key material, but our new key doesn't have a user ID yet, and therefore is not yet usable!
uid = pgpy.PGPUID.new('testname', comment='PGP Assignment', email='testname@abc.com')
# now we must add the new user id to the key. We'll need to specify all of our preferences at this point
# because PGPy doesn't have any built-in key preference defaults at this time
# this example is similar to GnuPG 2.1.x defaults, with no expiration or preferred keyserver
key.add_uid(uid, usage={KeyFlags.Sign, KeyFlags.EncryptCommunications, KeyFlags.EncryptStorage},
             hashes=[HashAlgorithm.SHA256, HashAlgorithm.SHA384, HashAlgorithm.SHA512, HashAlgorithm.SHA224],
             ciphers=[SymmetricKeyAlgorithm.AES256, SymmetricKeyAlgorithm.AES192, SymmetricKeyAlgorithm.AES128],
             compression=[CompressionAlgorithm.ZLIB, CompressionAlgorithm.BZ2, CompressionAlgorithm.ZIP, CompressionAlgorithm.Uncompressed])
# assuming we already have a primary key, we can generate a new key and add it as a subkey thusly:
subkey = pgpy.PGPKey.new(PubKeyAlgorithm.RSAEncryptOrSign, 4096)
# preferences that are specific to the subkey can be chosen here
# any preference(s) needed for actions by this subkey that not specified here
# will seamlessly "inherit" from those specified on the selected User ID
key.add_subkey(subkey, usage={KeyFlags.EncryptCommunications})
# ASCII armored. To share the key with a 3rd party.
#    with open('/dbfs/mnt/anuadlstest/private.asc', "w") as csv_file:
#      csv_file.write(str(key))
with open('/content/sample_data/publickey.asc', "w") as csv_file:
   csv_file.write(str(key.pubkey))
#print("Public key in string format :" + str(key.pubkey))
#print("Private key in string format :" + str(key))
string_bytes = str(key.pubkey).encode("ascii")
publicb64 = base64.b64encode(string_bytes)
print("Base64 encoded public key : " + str(publicb64))
string_bytes = str(key).encode("ascii")
privateb64 = base64.b64encode(string_bytes)
print("Base64 encoded private key : " + str(privateb64))

In [ ]:
#Encrypting a file using public key
import pgpy
from pgpy.constants import PubKeyAlgorithm, KeyFlags, HashAlgorithm, SymmetricKeyAlgorithm, CompressionAlgorithm
from timeit import default_timer as timer
import base64 
import io
 
KEY_PUB = base64.b64decode(publicb64).decode("ascii").lstrip()  
#print(KEY_PUB)

pub_key = pgpy.PGPKey()
pub_key.parse(KEY_PUB)
pass
# -READ THE FILE FROM MOUNT POINT-----------------
with io.open('/content/sample_data/california_housing_test.csv', "r",newline='') as csv_file:
    input_data = csv_file.read()                   # The io and newline retains the CRLF
    
t0 = timer()
#PGP Encryption start
msg = pgpy.PGPMessage.new(input_data)
###### this returns a new PGPMessage that contains an encrypted form of the original message
encrypted_message = pub_key.encrypt(msg)
pgpstr = str(encrypted_message)
with open('/content/sample_data/california_housing_test.csv.pgp', "w") as text_file:
    text_file.write(pgpstr)
print("Encryption Complete :" + str(timer()-t0))

In [ ]:
#Decrypting a file using private key
import pgpy
from pgpy.constants import PubKeyAlgorithm, KeyFlags, HashAlgorithm, SymmetricKeyAlgorithm, CompressionAlgorithm
from timeit import default_timer as timer
import base64 
import io
 
def get_private_key():
#    pk_base64 = dbutils.secrets.get(scope = "PGP", key = "pkbase64")
    pk_string = base64.b64decode(privateb64)
    pk_string = pk_string.decode("ascii")
    return str(pk_string)
 
private_key = get_private_key()
KEY_PRIV = private_key.lstrip()
 
priv_key = pgpy.PGPKey()
priv_key.parse(KEY_PRIV)
pass
 
#PGP Deryption start
t0 = timer()
message_from_file = pgpy.PGPMessage.from_file('/content/sample_data/california_housing_test.csv.pgp')
raw_message = priv_key.decrypt(message_from_file).message
with open('/content/sample_data/california_housing_test_pgpdecrypt.csv', "w") as csv_file:
    csv_file.write(raw_message)
print("Decryption Complete :" + str(timer()-t0))

In [ ]:
# This compares the original file with the decrypted file.
 
import sys
import hashlib
 
def hashfile(file):
    # A arbitrary (but fixed) buffer size (change accordingly)
    # 65536 = 65536 bytes = 64 kilobytes
    BUF_SIZE = 65536
    # Initializing the sha256() method
    sha256 = hashlib.sha256()
    # Opening the file provided as
    # the first commandline argument
    with open(file, 'rb') as f:
        while True:
            # reading data = BUF_SIZE from the file and saving it in a
            # variable
            data = f.read(BUF_SIZE)
            # True if eof = 1
            if not data:
                break
            # Passing that data to that sha256 hash function (updating the function with that data)
            sha256.update(data)
      
    # sha256.hexdigest() hashes all the input
    # data passed to the sha256() via sha256.update()
    # Acts as a finalize method, after which
    # all the input data gets hashed hexdigest()
    # hashes the data, and returns the output
    # in hexadecimal format
    return sha256.hexdigest()
 
# Calling hashfile() function to obtain hashes of the files, and saving the result in a variable
Initial_hash = hashfile('/content/sample_data/california_housing_test.csv')
Decrypted_hash = hashfile('/content/sample_data/california_housing_test_pgpdecrypt.csv')
  
# Doing primitive string comparison to
# check whether the two hashes match or not
if Initial_hash == Decrypted_hash:
    print("Both files are same")
    print(f"Hash: {Initial_hash}")
else:
    print("Files are different!")
    print(f"Hash of File 1: {Initial_hash}")
    print(f"Hash of File 2: {Decrypted_hash}")